In [3]:
import pandas as pd
df = pd.read_csv('../test_pred.csv')
df.columns

Index(['Unnamed: 0', 'issue_url', 'issue_label', 'issue_created_at',
       'issue_author_association', 'repository_url', 'issue_title',
       'issue_body', 'pred', 'encodings', 'features'],
      dtype='object')

In [4]:
df.iloc[0]

Unnamed: 0                                                                  6
issue_url                   https://api.github.com/repos/tlnagy/TIFF.jl/is...
issue_label                                                       enhancement
issue_created_at                                         2020-04-07T09:08:50Z
issue_author_association                                                 NONE
repository_url                    https://api.github.com/repos/tlnagy/TIFF.jl
issue_title                 ERROR: KeyError: key (TIFF.SAMPLEFORMAT_INT, 0...
issue_body                  One more error might need to be caught.\r\n`4D...
pred                                                                      bug
encodings                   {'input_ids': [101, 7561, 1024, 3145, 2121, 29...
features                                                        [1, 46, 0, 0]
Name: 0, dtype: object

In [30]:
from sklearn.metrics import confusion_matrix

C = confusion_matrix(df.issue_label, df.pred, labels = ['bug','enhancement', 'question'])

In [31]:
C.sum(axis=0) # col-wise

array([42062, 32427,  6029])

In [32]:
C.sum(axis=1) # row-wise

array([40288, 33203,  7027])

In [33]:
C

array([[36593,  2718,   977],
       [ 3570, 28647,   986],
       [ 1899,  1062,  4066]])

In [34]:
confusion_matrix(df.issue_label, df.pred, labels = ['bug','enhancement', 'question'],normalize='true') #Accuracy

array([[0.90828535, 0.06746426, 0.0242504 ],
       [0.1075204 , 0.86278348, 0.02969611],
       [0.27024335, 0.15113135, 0.5786253 ]])

In [39]:
import sklearn
y_true = df.issue_label
y_pred = df.pred
for label in ["bug", "enhancement", "question"]:
  P_c = sklearn.metrics.precision_score(y_true, y_pred, average=None, labels=[label])[0]
  R_c = sklearn.metrics.recall_score(y_true, y_pred, average=None, labels=[label])[0]
  F1_c = sklearn.metrics.f1_score(y_true, y_pred, average=None, labels=[label])[0]
  print(f"=*= {label} =*=")
  print(f"precision:\t{P_c:.4f}")
  print(f"recall:\t\t{R_c:.4f}")
  print(f"F1 score:\t{F1_c:.4f}")
  print()


P = sklearn.metrics.precision_score(y_true, y_pred, average='micro')
R = sklearn.metrics.recall_score(y_true, y_pred, average='micro')
F1 = sklearn.metrics.f1_score(y_true, y_pred, average='micro')

print("=*= global =*=")
print(f"precision:\t{P:.4f}")
print(f"recall:\t\t{R:.4f}")
print(f"F1 score:\t{F1:.4f}")

=*= bug =*=
precision:	0.8700
recall:		0.9083
F1 score:	0.8887

=*= enhancement =*=
precision:	0.8834
recall:		0.8628
F1 score:	0.8730

=*= question =*=
precision:	0.6744
recall:		0.5786
F1 score:	0.6229

=*= global =*=
precision:	0.8608
recall:		0.8608
F1 score:	0.8608


In [42]:
filtered_df = df[(df.issue_label != df.pred) & (df.issue_label == 'question')] 

In [45]:
filtered_df.shape

(2961, 11)

In [66]:
filtered_df['isquestion'] = filtered_df.features.apply(lambda x: int(x.split(',')[-1][1:-1]))
filtered_df.iloc[0]

/home/antpc/miniconda3/envs/tree/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


Unnamed: 0                                                                529
issue_url                   https://api.github.com/repos/AlexxIT/YandexSta...
issue_label                                                          question
issue_created_at                                         2020-11-25T14:32:40Z
issue_author_association                                                 NONE
repository_url              https://api.github.com/repos/AlexxIT/YandexSta...
issue_title                                                      Не восприним
issue_body                  Помогите плз.\r\n```\r\n  action:\r\n  - data:...
pred                                                                      bug
encodings                   {'input_ids': [101, 1192, 15290, 1182, 14150, ...
features                                                        [1, 49, 0, 0]
isquestion                                                                  0
Name: 43, dtype: object

In [70]:
filtered_df.isquestion.sum()/filtered_df.shape[0]

0.10064167510976021

In [71]:
question_df = df[(df.issue_label == 'question')]
question_df['isquestion'] = question_df.features.apply(lambda x: int(x.split(',')[-1][1:-1]))

/home/antpc/miniconda3/envs/tree/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [73]:
print(question_df.shape)
question_df.isquestion.sum()/question_df.shape[0]

(7027, 12)


0.24847018642379393

In [74]:
correct_question_df = df[(df.issue_label == df.pred) & (df.issue_label == 'question')] 
print(correct_question_df.shape)
correct_question_df['isquestion'] = correct_question_df.features.apply(lambda x: int(x.split(',')[-1][1:-1]))
correct_question_df.isquestion.sum()/correct_question_df.shape[0]

(4066, 11)


/home/antpc/miniconda3/envs/tree/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


0.3561239547466798

In [75]:
filtered_df.shape

(2961, 12)

In [62]:
int(R['features'].split(',')[-1][1:-1])

0

In [76]:
c=0
for i,R in filtered_df.iterrows():
    print(R['issue_title'])
    print('--'*20)
    print(R['issue_body'])
    print('--'*20)
    print(R['features'])
    print('Prediction:',R['pred'])
    print('=='*20)
    if c==100:
        break
    c+=1

Не восприним
----------------------------------------
Помогите плз.
```
  action:
  - data:
      message: Двигатель запущен. Температура  {{ states("sensor.1077197514_engine_temperature")|round  }}
    service: notify.mobile_app_iphone_almazbek
  - data:
      entity_id:
      - media_player.yandex_station_04007884c9143c1709cd
      volume_level: 0.8
    service: media_player.volume_set
  - data:
      media_content_id: Двигатель запущен. Температура  {{ states("sensor.1077197514_engine_temperature")|round  }}
      media_content_type: text
    entity_id: media_player.yandex_station_04007884c9143c1709cd
    service: media_player.play_media
```
Push приходит на телефон с температурой, а Алиса молчит
----------------------------------------
[1, 49, 0, 0]
Prediction: bug
Using type declarations - Unable to locate entry point
----------------------------------------
## Description

I just sent a [PR to upgrade Chart.js to 0.20.9](https://github.com/chartjs/Chart.js/pull/8267)! :tada: 

Ch